# Molecular Foundry User Meeting 2024
This notebook describes how to explore of image stacks
 1. Reading large volumes 📚
 2. Image enhancing techniques 💎
 3. Thresholding techniques ☯
 4. Visualization 🧿

Created by Dani Ushizima, CAMERA, LBNL - Jul 1st 2024


🪨 Motivation behind data 🔬
- **nanoCT**: basalt rock sample
- **why**? Basalt rock samples are highly relevant to the sequestration of atmospheric CO2 through a process known as mineral carbonation
- *courtesy: Arun Bhattacharjee*

In [1]:
%matplotlib inline

In [2]:
import numpy as np

import matplotlib.pyplot as plt

from skimage import img_as_ubyte, filters, morphology, io
from skimage.filters import threshold_isodata

from glob import glob
from sys import getsizeof
import fnmatch,os

# 1.Read an image stack 📚
- from Google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
datapath = "/content/drive/My Drive/Colab Notebooks/tutorial/2024_microCT_tutorial/data/" #change to your data path
!ls -lt "$datapath"

In [ ]:

nanopath = datapath + "rock_nanoCT/" #change to your data path

In [ ]:
def count_files_and_size(directory):
  """Counts the number of files and total size in a directory.
  """
  total_size = 0
  for filename in os.listdir(directory):
    filepath = os.path.join(directory, filename)
    if os.path.isfile(filepath):
      total_size += os.path.getsize(filepath)
  num_files = len(glob(directory + "/*"))
  total_size = round(total_size / 1024 / 1024, 2)
  print(f"Number of files in directory: {num_files}")
  print(f"Total size of files in directory: {total_size} MB")
  #return num_files, round(total_size / 1024 / 1024, 2)


In [ ]:
count_files_and_size(nanopath)

In [ ]:
#Load all file names
files = glob(nanopath+'*')
files.sort()


In [ ]:
#Describe importante info of an image
def describe_image(image):
  print('-----------------------------------------------------------------------')
  print('Image shape is ',image.shape)
  print("Image size in mem: {}MB".format(round(getsizeof(image) / 1024 / 1024,2)))
  print('@CenterSlice: min=',image.min(),',mean=',np.around(image.mean(),decimals=2),',max=',image.max())
  print('dtype = ',image.dtype)
  print('-----------------------------------------------------------------------')

In [ ]:
image = io.imread(files[0])

In [ ]:
describe_image(image)
plt.imshow(image,cmap='gray')
plt.title('Slice #0 - full resolution')


## 1.1. Quick visualizations
- one slice at a time
- equally spaced slices
- downsample
- montage
- plotly

In [ ]:
def montage(path,extension="*.tif*",downsample=10,save=False):
        '''Create montage 3x3
        '''
        files = glob(nanopath+extension)
        files.sort()
        nfiles = len(files)
        fig, axes = plt.subplots(3, 3, figsize=(10, 10))
        #Plotting subset of slices evenly spaced
        islices = np.linspace(0,nfiles-1,9,dtype=int)
        islices = islices.astype(int)
        for i, n in enumerate(islices):
            img = io.imread(files[n])
            img = img[::downsample,::downsample]
            xy = np.unravel_index(i, (3,3))
            axes[xy].imshow(img,cmap='gray')
            axes[xy].set_title('Slice %i'% n)
            axes[xy].set_axis_off()
            #fig.subplots_adjust( wspace = -0.1,  hspace = 0.1 )
        if(save):
            fig.savefig(path+'MyMontage.png')
            print('Saved@ '+path+'MyMontage.png')

In [ ]:
# quick pick at 32 GB experiment
montage(nanopath)

In [ ]:
import plotly.express as px
from skimage import io

def create_image_animation(fnames, downsample=10, nframes=10):
    """
    Create an animated plot of images, evenly spaced and downsampled.
    """
    nfiles = len(fnames)

    if nfiles > nframes:
        # Select indices for evenly z-spaced images
        islices = np.linspace(0, nfiles - 1, num=nframes, dtype=int)

        # Load and downsample images
        limg = []
        for n in islices:
            img = io.imread(fnames[n])
            img_downsampled = img[::downsample, ::downsample]
            limg.append(img_downsampled)

        # Stack the images for animation if not empty
        if limg:
            vimg = np.stack(limg)
            fig = px.imshow(vimg, animation_frame=0, labels=dict(animation_frame="Image Index"))
            fig.update_layout(height=600)  # Adjust layout as needed
            fig.show()
        else:
            print("No images loaded or downsampled images are empty.")
    else:
        print('Not enough files - change nframes or provide more files.')
    return limg


In [ ]:
#Create animation and return a list with nframes across stack
listImg = create_image_animation(files)

##1.2.Check histogram
- what's the range?
- are the intensity levels uniform across slices?

In [ ]:
from ipywidgets import interact, IntSlider
from IPython.display import clear_output

def seeHistSlice(img, nslice):
    '''Interactively choose a particular slice and see the histogram
    '''
    clear_output(wait=True)  # Clear the output to avoid stacking plots
    subimage = img[nslice, :, :]
    hist, bins = np.histogram(subimage, bins=50)  # Adjust bins or range as needed

    fig, ax = plt.subplots(ncols=2, figsize=(12, 5))  # Adjust figure size as needed

    # Image display
    ax[0].imshow(subimage, interpolation='nearest', cmap=plt.cm.gray)
    ax[0].axis('off')
    ax[0].set_title('Slice #' + str(nslice))

    # Histogram display
    ax[1].bar(bins[:-1], hist, width=np.diff(bins), edgecolor='black', align='edge')
    ax[1].set_title('Histogram of gray values')
    ax[1].grid(True)
    plt.tight_layout()
    plt.show()

def slicingHist(img_stack):
    def slicer(z):
        seeHistSlice(img_stack, z)
    interact(slicer, z=IntSlider(min=0, max=len(img_stack)-1, step=1, value=len(img_stack)//2), description='Slice Index')


In [ ]:
fullstack = np.stack(listImg)
slicingHist(fullstack)

# 2.Image enhancing techniques 💎
- denoising: median, bilateral
- sharpening
- morphological operators
- differential operators

In [ ]:
def imshowcmp(before,after,lut='plasma'):
    '''Show 2 images side by side'''
    f, ax = plt.subplots(1, 2, figsize=(10, 10))
    ax[0].imshow(before,cmap=lut)
    ax[0].set_title('before')
    ax[1].imshow(after,cmap=lut)
    ax[1].set_title('after')

### 2.1.Filtering techniques to improve segmentation

In [ ]:
#select one slice and either downsample or crop to a small matrix
downsample = 10
aslice = image[::10,::10] #downsampling

In [ ]:
# MEDIAN
selem = morphology.diamond(3)
medianImage = filters.median(aslice,selem)
imshowcmp(aslice,medianImage,'gray')

In [ ]:
# CONTRAST ENHANCEMENT
sharpImage = filters.unsharp_mask(aslice, radius=20, amount=1)
imshowcmp(aslice,sharpImage,'gray')

In [ ]:
# MORPHOLOGICAL OPERATORS
dilatImage = morphology.dilation(aslice,selem)
imshowcmp(aslice,dilatImage,'gray')

In [ ]:
#This will give you an error if image doesn't have pixel depth = 8:
# Possible precision loss converting image of type float32 to uint8 as required by rank filters.
#  Convert manually using skimage.util.img_as_ubyte to silence this warning.

bilatImage = filters.rank.mean_bilateral(aslice,selem)
imshowcmp(aslice,bilatImage,'gray')

In [ ]:
# If your image is not in the range
#Normalize the image to the range [0, 1], then turn into uint8, then apply filter
aslice_normalized = (aslice - aslice.min()) / (aslice.max() - aslice.min())
aslice_normalized = img_as_ubyte(aslice_normalized)
bilatImage = filters.rank.mean_bilateral(aslice_normalized,selem)
imshowcmp(aslice,bilatImage,'gray')


In [ ]:
# EDGE DETECTION
sobelImage = filters.sobel(aslice)
imshowcmp(aslice,sobelImage,'gray')

# 3.Thresholding for segmentation ☯
- rough partition of image if bimodal
- useful:
  - for visualization
  - for annotating data before ML


In [ ]:
t= filters.try_all_threshold(aslice, figsize=(8,8), verbose=False)
#plt.savefig("thresholdAll.png", bbox_inches='tight')

## Combining operators

In [ ]:
import numpy as np

def circular_crop(image, center, radius):
  """
  Crops a circular region from an image, filling the outside with black.

  Args:
    image: The input image as a NumPy array.
    center: A tuple (x, y) specifying the center of the circle.
    radius: The radius of the circle.

  Returns:
    The cropped image as a NumPy array.
  """

  x, y = np.indices(image.shape)
  distance_from_center = np.sqrt((x - center[0])**2 + (y - center[1])**2)
  mask = distance_from_center <= radius

  cropped_image = np.zeros_like(image)
  cropped_image[mask] = image[mask]

  return cropped_image


In [ ]:
# Creating a fast segmentation algorithm
from scipy.ndimage import binary_fill_holes

aslice = image
img2 = filters.median(aslice, morphology.diamond(3))
t=filters.threshold_isodata(img2) #calculates the threshold
binary = img2>t #apply to image
binary = morphology.closing(binary,selem)
binary = morphology.remove_small_objects(binary, 100)
binary = binary_fill_holes(binary)
binary = circular_crop(binary, (1024, 1024), 750)

imshowcmp(aslice,binary,'gray')

#4.Volumetric visualization 🧿
📣 Learn about you data first: 📣
* size
* pixel depth
* most relevant slices
* what to look first, e.g., general view to detail

<font color="blue">
So what did we learn:
a) 32 GB, so downsample
b) cylinder around, so mask it out to see inside
c) 30% of bottom slices are less relevant
</font>


In [ ]:
def fastSegmentation(aslice,center=(10,10), radius=10):
  img2 = filters.median(aslice, morphology.diamond(3))
  t=filters.threshold_isodata(img2) #calculates the threshold
  binary = img2>t #apply to image
  binary = morphology.closing(binary,selem)
  binary = morphology.remove_small_objects(binary, 100)
  binary = binary_fill_holes(binary)
  binary = circular_crop(binary, center, radius)
  return binary

In [ ]:
import plotly.express as px
from skimage import io

def readStack(fnames, downsample=10, nframes=20, center=(10,10), radius=10, segmented=False):
    """
    Read a subset of stack slices evenly spaced, downsampled and cropped by circle.
    """
    nfiles = len(fnames)

    if nfiles > nframes:
        # Select indices for evenly z-spaced images
        islices = np.linspace(0, nfiles - 1, num=nframes, dtype=int)

        # Load and downsample images
        limg = []
        for n in islices:
            img = io.imread(fnames[n])
            img = img[::downsample, ::downsample]
            img_downsampled = (img - img.min()) / (img.max() - img.min())
            img_downsampled = img_as_ubyte(img_downsampled)
            if segmented:
                cc = circular_crop(img_downsampled, center, radius)
            else:
                cc = fastSegmentation(img_downsampled, center, radius)
            limg.append(cc)

        # Stack the images for animation if not empty
        if limg:
            vimg = np.stack(limg)
        else:
            print("No images loaded or downsampled images are empty.")
    else:
        print('Not enough files - change nframes or provide more files.')
    return vimg

In [ ]:
n = len(files)
nKey = n - round(n*0.3) #ignoring bottom slices
onlyKeyFiles = files[:nKey]
bigLimg = readStack(onlyKeyFiles, downsample=10,nframes=100, center=(102, 102), radius=75)

In [10]:
import tifffile

# Save the variable to a TIFF file
tifffile.imwrite('checkPointBigLimg.tif', bigLimg)

# Read the TIFF file back
loaded_bigLimg = tifffile.imread('checkPointBigLimg.tif')


NameError: name 'bigLimg' is not defined

In [ ]:
plt.imshow(bigLimg[0],cmap='gray')

In [ ]:
#Visualization

In [ ]:
!pip install imjoy-elfinder --quiet
!pip install imjoy --quiet
!pip install -U imjoy-jupyter-extension numpy scikit-image --quiet
!pip install itk --quiet
!pip install 'itkwidgets>=1.0a23' --quiet

import itk
from itkwidgets import view

In [ ]:
viewer = view(bigLimg, slicing_planes=True)
viewer